# Playground for cause and effect two sentences

TODO:explain & explore

In [18]:
import os
import numpy as np
import openai
import pandas as pd
from tqdm import tqdm
from setups.ce_two_sentences_setup import CETwoSentences

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_KEY")

In [19]:
gpt3_model = "text-davinci-001"
#gpt3_model = "text-curie-001"
#gpt3_model = "text-babbage-001"
#gpt3_model = "text-ada-001"

In [20]:
# test run
def gpt3_response(prompt, model):
    response = openai.Completion.create(engine=model, prompt=prompt, max_tokens=20, temperature=0)
    return(response["choices"][0]["text"])

In [21]:
toy_problem_3_colors = ToyProblem3Colors()

In [22]:
toy_problem_3_colors.get_random_subsets(3)

array([['blue', 'red', 'brown'],
       ['purple', 'black', 'white'],
       ['red', 'green', 'white']], dtype='<U6')

In [23]:
print(toy_problem_3_colors.generate_single_sequence(0, switched=False))
print(toy_problem_3_colors.generate_single_sequence(0, switched=True))

The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole.
The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole.


In [24]:
toy_problem_3_colors.generate_sequences_df()

,sequence,switched,first_color,second_color,final_color
0,The blue ball hit the red ball. The red ball h...,False,blue,red,green
1,The red ball hit the green ball. The blue ball...,True,blue,red,green
2,The blue ball hit the red ball. The red ball h...,False,blue,red,brown
3,The red ball hit the brown ball. The blue ball...,True,blue,red,brown
4,The blue ball hit the red ball. The red ball h...,False,blue,red,purple
...,...,...,...,...,...
65,The purple ball hit the white ball. The brown ...,True,brown,purple,white
66,The brown ball hit the black ball. The black b...,False,brown,black,white
67,The black ball hit the white ball. The brown b...,True,brown,black,white
68,The purple ball hit the black ball. The black ...,False,purple,black,white


In [26]:
toy_problem_3_colors.save_sequences_df("data/toy_problem_3_colors/toy_problem_3c.csv")

### Zero shot

In [27]:
prompt_zs_first = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="first")
prompt_zs_second = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="second")
prompt_zs_final = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="final")

prompt_zs_first_switched = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="first")
prompt_zs_second_switched = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="second")
prompt_zs_final_switched = toy_problem_3_colors.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="final")

print(prompt_zs_first + gpt3_response(prompt_zs_first, gpt3_model) + "\n")
print(prompt_zs_second + gpt3_response(prompt_zs_second, gpt3_model) + "\n")
print(prompt_zs_final + gpt3_response(prompt_zs_final, gpt3_model) + "\n")
print(prompt_zs_first_switched + gpt3_response(prompt_zs_first_switched, gpt3_model) + "\n")
print(prompt_zs_second_switched + gpt3_response(prompt_zs_second_switched, gpt3_model) + "\n")
print(prompt_zs_final_switched + gpt3_response(prompt_zs_final_switched, gpt3_model) + "\n")

The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 

The blue ball started the chain.

The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball was second in the chain? Answer: 

The blue ball was second in the chain.

The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball fell into the hole? Answer: 

The green ball fell into the hole.

The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 

The blue ball started the chain.

The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball was second in the chain? Answer: 

The blue ball was second in the chain.

The red ball hit the green ball. The blue ball hit the red ball

In [8]:
print(toy_problem_3_colors.generate_all_prompts_zero_shot(question='first')[:3])
print(toy_problem_3_colors.generate_all_prompts_zero_shot(question='second')[:3])
print(toy_problem_3_colors.generate_all_prompts_zero_shot(question='final')[:3])

['The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: ', 'The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: ', 'The blue ball hit the red ball. The red ball hit the brown ball. The brown ball fell in the hole. Question: Which ball started the chain? Answer: ']
['The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball was second in the chain? Answer: ', 'The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball was second in the chain? Answer: ', 'The blue ball hit the red ball. The red ball hit the brown ball. The brown ball fell in the hole. Question: Which ball was second in the chain? Answer: ']
['The blue ball hit the red ball. The red ball hit the green ball. The green bal

### One shot

In [28]:
prompt_os_first = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=False, question="first")
prompt_os_second = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=False, question="second")
prompt_os_final = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=False, question="final")

prompt_os_first_switched = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=True, question="first")
prompt_os_second_switched = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=True, question="second")
prompt_os_final_switched = toy_problem_3_colors.generate_single_prompt_one_shot(n_subset=0, switched=True, question="final")

print(prompt_os_first + gpt3_response(prompt_os_first, gpt3_model) + "\n")
print(prompt_os_second + gpt3_response(prompt_os_second, gpt3_model) + "\n")
print(prompt_os_final + gpt3_response(prompt_os_final, gpt3_model) + "\n")
print(prompt_os_first_switched + gpt3_response(prompt_os_first_switched, gpt3_model) + "\n")
print(prompt_os_second_switched + gpt3_response(prompt_os_second_switched, gpt3_model) + "\n")
print(prompt_os_final_switched + gpt3_response(prompt_os_final_switched, gpt3_model) + "\n")

The red ball hit the brown ball. The brown ball hit the white ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the red ball
The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 

The blue ball

The red ball hit the brown ball. The brown ball hit the white ball. The white ball fell in the hole. Question: Which ball was second in the chain? Answer: the brown ball
The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball was second in the chain? Answer: 

The blue ball

The purple ball hit the white ball. The blue ball hit the purple ball. The white ball fell in the hole. Question: Which ball fell into the hole? Answer: the white ball
The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball fell into the hole? Answer: 

The green ball

The

In [15]:
print(toy_problem_3_colors.generate_all_prompts_one_shot(question='first')[:3])
print("\n")
print(toy_problem_3_colors.generate_all_prompts_one_shot(question='second')[:3])
print("\n")
print(toy_problem_3_colors.generate_all_prompts_one_shot(question='final')[:3])

['The red ball hit the green ball. The green ball hit the purple ball. The purple ball fell in the hole. Question: Which ball started the chain? Answer: the red ball\nThe blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: ', 'The brown ball hit the purple ball. The purple ball hit the black ball. The black ball fell in the hole. Question: Which ball started the chain? Answer: the brown ball\nThe red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: ', 'The purple ball hit the black ball. The red ball hit the purple ball. The black ball fell in the hole. Question: Which ball started the chain? Answer: the red ball\nThe blue ball hit the red ball. The red ball hit the brown ball. The brown ball fell in the hole. Question: Which ball started the chain? Answer: ']


['The blue ball hit the red ball. The red ball h

### K=5 shot

In [29]:
prompt_ks_first = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=False, question="first")
prompt_ks_second = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=False, question="second")
prompt_ks_final = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=False, question="final")

prompt_ks_first_switched = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=True, question="first")
prompt_ks_second_switched = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=True, question="second")
prompt_ks_final_switched = toy_problem_3_colors.generate_single_prompt_k_shot(n_subset=0, k=5, switched=True, question="final")

print(prompt_ks_first + gpt3_response(prompt_ks_first, gpt3_model) + "\n")
print(prompt_ks_second + gpt3_response(prompt_ks_second, gpt3_model) + "\n")
print(prompt_ks_final + gpt3_response(prompt_ks_final, gpt3_model) + "\n")
print(prompt_ks_first_switched + gpt3_response(prompt_ks_first_switched, gpt3_model) + "\n")
print(prompt_ks_second_switched + gpt3_response(prompt_ks_second_switched, gpt3_model) + "\n")
print(prompt_ks_final_switched + gpt3_response(prompt_ks_final_switched, gpt3_model) + "\n")

The green ball hit the black ball. The blue ball hit the green ball. The black ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball
The blue ball hit the red ball. The red ball hit the purple ball. The purple ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball
The blue ball hit the green ball. The green ball hit the brown ball. The brown ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball
The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball
The blue ball hit the red ball. The red ball hit the white ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball
The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 

the red ball

The red ball hit the brown b

In [16]:
print(toy_problem_3_colors.generate_all_prompts_k_shot(k=5, question='first')[:3])
print("\n")
print(toy_problem_3_colors.generate_all_prompts_k_shot(k=5, question='second')[:3])
print("\n")
print(toy_problem_3_colors.generate_all_prompts_k_shot(k=5, question='final')[:3])

['The green ball hit the white ball. The red ball hit the green ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the red ball\nThe green ball hit the brown ball. The brown ball hit the white ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the green ball\nThe blue ball hit the red ball. The red ball hit the brown ball. The brown ball fell in the hole. Question: Which ball started the chain? Answer: the blue ball\nThe black ball hit the white ball. The purple ball hit the black ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the purple ball\nThe red ball hit the brown ball. The brown ball hit the white ball. The white ball fell in the hole. Question: Which ball started the chain? Answer: the red ball\nThe blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: ', 'The green ball hit the pu